In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
import matplotlib.pyplot as plt
%matplotlib inline

#### The data used

In [61]:
# #df = pd.read_csv('data/anemia_synth_dataset_hb_some_nans.csv')
# df = pd.read_csv('data/anemia_synth_dataset_with_unspecified.csv')
# df = df.fillna(0)
# # classes = list(df.label.unique())
# # nums = [i for i in range(len(classes))]
# # class_dict = dict(zip(classes, nums))
# class_dict = {'No anemia': 0, 'Hemolytic anemia': 1, 'Aplastic anemia': 2, 'Iron deficiency anemia': 3,
#               'Vitamin B12/Folate deficiency anemia': 4, 'Anemia of chronic disease': 5, 'Unspecified anemia': 6}
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [62]:
X_test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = X_test_df.iloc[:, 0:-1]
y_test = X_test_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [63]:
X_test.shape, y_test.shape

((14000, 17), (14000,))

In [65]:
test_df =pd.read_csv('../../final/test_dfs/dqn_test_df_basic_5000000.csv')
test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,5.0,0.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",5.0,5.0
1,5.0,1.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",1.0,1.0
2,6.0,2.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 't...",4.0,4.0
3,3.0,3.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'No anemia']",0.0,0.0
4,5.0,4.0,1.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",7.0,7.0


In [66]:
def get_min_of_array(array, n):
    '''The n least elements in an array'''
    idx = np.argpartition(array, n)
    min_vals = array[idx[:n]]
    return min_vals

In [67]:
# X_test_df = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test)], axis=1)
# X_test_df.columns = df.columns
# X_test_df.head()

In [68]:
a1 = np.array(test_df['y_actual'], dtype=np.float32)
a2 = np.array(X_test_df['label'], dtype=np.float32)
assert np.array_equal(a1, a2)

#### 0 - No Anemia

In [12]:
no_test_df = test_df[test_df.y_actual== 0]
no_x_test_df = X_test_df[X_test_df.label==0]
combined_no_test_df = pd.concat([no_test_df, no_x_test_df], axis=1)
combined_no_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
20,2.0,20.0,1.0,1.0,0.0,"['hematocrit', 'No anemia']",0.0,0.0,14.357527,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,43.072581,-1.00000,0
41,3.0,41.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'No anemia']",0.0,0.0,12.980411,-1.000000,...,0,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,38.941233,-1.00000,0
50,3.0,50.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'No anemia']",0.0,0.0,12.508738,-1.000000,...,0,1.580692,26.953005,-1.000000,-1.00000,-1.000000,-1.000000,37.526213,-1.00000,0
70,2.0,70.0,1.0,1.0,0.0,"['hematocrit', 'No anemia']",0.0,0.0,13.751608,-1.000000,...,1,1.841386,71.188182,-1.000000,-1.00000,-1.000000,-1.000000,41.254824,-1.00000,0
72,2.0,72.0,1.0,1.0,0.0,"['hematocrit', 'No anemia']",0.0,0.0,13.983326,183.693645,...,1,1.623076,47.526064,93.465071,48.54357,28.924289,119.257847,41.949977,110.40971,0


In [13]:
combined_no_test_df.y_pred.value_counts()

0.0    1934
7.0      34
5.0      11
3.0       8
6.0       7
1.0       6
Name: y_pred, dtype: int64

In [15]:
df_0 = combined_no_test_df[combined_no_test_df.y_pred==5]
df_0.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
1110,5.0,1110.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.067067,133.074973,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.201201,-1.000000,0
4822,5.0,4822.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.061731,115.513088,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.185193,-1.000000,0
5043,5.0,5043.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.084466,-1.000000,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.253398,-1.000000,0
5627,5.0,5627.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.003728,452.696616,...,0,0.445645,39.646397,-1.000000,-1.000000,-1.000000,-1.000000,36.011183,-1.000000,0
6022,5.0,6022.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ret_count', '...",0.0,5.0,12.001263,175.160936,...,0,1.531779,57.080536,68.971914,42.617873,18.643214,46.295387,36.003790,70.547835,0


In [16]:
df_0.describe()[['hemoglobin', 'gender', 'mcv', 'ret_count']]

,hemoglobin,gender,mcv,ret_count
count,11.000000,11.000000,11.000000,11.000000
mean,12.224520,0.181818,91.274820,3.935787
std,0.399814,0.404520,5.944908,1.309184
min,12.001263,0.000000,80.137404,2.336610
25%,12.026071,0.000000,87.465447,3.032254
50%,12.067067,0.000000,92.892681,3.817957
75%,12.077971,0.000000,95.374395,4.809621
max,13.035286,1.000000,98.623839,5.992682


#### 1 - Vitamin b12

In [17]:
vit_test_df = test_df[test_df.y_actual == 1]
vit_x_test_df = X_test_df[X_test_df.label== 1]
combined_vit_test_df = pd.concat([vit_test_df, vit_x_test_df], axis=1)
combined_vit_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
12,5.0,12.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,8.363249,-1.000000,...,0,1.066354,24.964254,117.646559,-1.000000,6.393712,-1.000000,25.089747,-1.000000,1
19,5.0,19.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,12.647502,277.291915,...,1,0.867387,51.916799,108.418585,73.838946,7.631822,79.222036,37.942505,51.190298,1
39,5.0,39.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,8.855406,-1.000000,...,0,1.131964,143.711309,-1.000000,3.137595,-1.000000,73.771704,26.566218,99.359444,1
45,5.0,45.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,9.734492,399.066382,...,0,0.549935,25.665206,129.592979,30.180260,19.843242,69.527568,29.203475,81.878709,1
48,5.0,48.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,1.0,9.966403,127.222346,...,0,1.382820,13.616328,-1.000000,-1.000000,-1.000000,-1.000000,29.899208,-1.000000,1


In [18]:
combined_vit_test_df.y_pred.value_counts()

1.0    1795
7.0       4
2.0       2
Name: y_pred, dtype: int64

In [21]:
df_1 = combined_vit_test_df[combined_vit_test_df.y_pred==7]
df_1.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
4029,6.0,4029.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,12.957460,437.537271,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,38.872380,-1.000000,1
5252,6.0,5252.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,6.345608,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,19.036825,-1.000000,1
7874,6.0,7874.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,6.334332,286.380455,...,0,0.214912,93.250973,105.229261,48.244279,7.533844,73.487029,19.002996,14.164076,1
9230,6.0,9230.0,0.0,-1.0,1.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",1.0,7.0,11.401385,-1.000000,...,1,0.411344,97.226823,-1.000000,-1.000000,-1.000000,-1.000000,34.204156,-1.000000,1


In [22]:
df_1.describe()[['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils']]

,hemoglobin,gender,mcv,segmented_neutrophils
count,4.000000,4.00,4.000000,4.000000
mean,9.259696,0.75,103.128866,0.142340
std,3.430741,0.50,0.951668,0.059876
min,6.334332,0.00,101.863661,0.101124
25%,6.342789,0.75,102.682136,0.108956
50%,8.873497,1.00,103.322235,0.118686
75%,11.790404,1.00,103.768965,0.152070
max,12.957460,1.00,104.007335,0.230863


In [23]:
# a4= vit_test_df[vit_test_df.y_actual != vit_test_df.y_pred]
# a4.head()

#### 2 - Unspecified Anemia

In [24]:
unspec_test_df = test_df[test_df.y_actual == 2]
unspec_x_test_df = X_test_df[X_test_df.label== 2]
combined_unspec_test_df = pd.concat([unspec_test_df, unspec_x_test_df], axis=1)
combined_unspec_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
5,5.0,5.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,6.650576,96.865214,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,19.951727,-1.000000,2
6,5.0,6.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,7.448639,392.097724,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.345916,-1.000000,2
8,5.0,8.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,12.111603,-1.000000,...,1,1.083396,134.192397,87.526295,51.076357,13.526479,93.942577,36.334809,82.152699,2
18,5.0,18.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,6.387051,219.772768,...,1,1.794993,45.872403,87.511535,-1.000000,19.533982,-1.000000,19.161152,-1.000000,2
25,5.0,25.0,1.0,1.0,0.0,"['hematocrit', 'gender', 'mcv', 'segmented_neu...",2.0,2.0,10.492080,94.784726,...,0,-1.000000,-1.000000,69.532226,72.191666,29.101395,137.509264,31.476239,15.761581,2


In [25]:
combined_unspec_test_df.y_pred.value_counts()

2.0    1793
Name: y_pred, dtype: int64

In [27]:
df_2 = combined_unspec_test_df[combined_unspec_test_df.y_pred==0]
df_2.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label


In [28]:
df_2.describe()[['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils']]

,hemoglobin,gender,mcv,segmented_neutrophils
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


#### 5 - Hemolytic Anemia

In [95]:
hemolytic_test_df = test_df[test_df.y_actual == 5]
hemolytic_x_test_df = X_test_df[X_test_df.label== 5]
combined_hemolytic_test_df = pd.concat([hemolytic_test_df, hemolytic_x_test_df], axis=1)
combined_hemolytic_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,5.0,0.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",5.0,5.0,7.116363,-1.000000,...,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
5,5.0,5.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",5.0,5.0,11.593572,223.938975,...,1,0.706771,145.955520,120.743927,0.543031,26.376968,114.959980,34.780716,69.114895,5
7,5.0,7.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",5.0,5.0,12.987288,-1.000000,...,1,1.211434,100.505296,-1.000000,-1.000000,-1.000000,-1.000000,38.961863,-1.000000,5
14,7.0,14.0,0.0,-1.0,0.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",5.0,2.0,10.896503,117.880415,...,0,1.763945,135.353641,117.048871,75.555430,28.472655,76.407250,32.689509,56.041269,5
23,5.0,23.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",5.0,5.0,8.251009,171.751134,...,1,0.373474,138.806059,-1.000000,-1.000000,-1.000000,-1.000000,24.753026,-1.000000,5


In [96]:
combined_hemolytic_test_df.y_pred.value_counts()

5.0    1536
7.0     151
1.0      49
6.0      34
3.0      27
2.0       6
4.0       2
Name: y_pred, dtype: int64

In [97]:
df_5 = combined_hemolytic_test_df[combined_hemolytic_test_df.y_pred==7]
df_5.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
49,5.0,49.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",5.0,7.0,6.474492,-1.000000,...,0,0.737623,103.590937,-1.000000,-1.000000,-1.000000,-1.00000,19.423475,-1.000000,5
90,5.0,90.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",5.0,7.0,6.115602,365.799899,...,0,1.413737,73.339646,54.324384,7.162349,5.328433,94.45921,18.346806,157.387303,5
102,5.0,102.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",5.0,7.0,11.359526,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,34.078577,-1.000000,5
218,7.0,218.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",5.0,7.0,7.941232,127.143493,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,23.823696,-1.000000,5
227,7.0,227.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",5.0,7.0,8.583652,-1.000000,...,0,0.829701,132.471574,103.649357,31.942823,4.426256,107.14336,25.750956,-1.000000,5


In [98]:
df_5['trajectory'].value_counts()

['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'Inconclusive diagnosis']                                              85
['gender', 'hematocrit', 'mcv', 'ferritin', 'Inconclusive diagnosis']                                                           29
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'rbc', 'tibc', 'Inconclusive diagnosis']                               25
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'rbc', 'tibc', 'ret_count', 'Inconclusive diagnosis']                   4
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'rbc', 'tibc', 'ret_count', 'glucose', 'Inconclusive diagnosis']        3
['gender', 'hematocrit', 'mcv', 'ferritin', 'tibc', 'Inconclusive diagnosis']                                                    2
['gender', 'hematocrit', 'hemoglobin', 'mcv', 'ferritin', 'Inconclusive diagnosis']                                              1
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'ret_count', 'serum_iron',

In [81]:
import ast
df_5['trajectory'] = df_5['trajectory'].apply(lambda row: ast.literal_eval(row))
df_5.iloc[0]['trajectory']

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



['gender',
 'hematocrit',
 'mcv',
 'segmented_neutrophils',
 'Inconclusive diagnosis']

In [93]:
a = pd.DataFrame()
for i, row in df_5.iterrows():
#     print(row.trajectory)
    if 'ferritin' in row.trajectory:
        a = a.append(row)
a.head()

,cholestrol,copper,creatinine,episode_length,ethanol,ferritin,folate,gender,glucose,hematocrit,...,ret_count,reward,segmented_neutrophils,serum_iron,terminated,tibc,trajectory,tsat,y_actual,y_pred
960,112.181291,117.252448,0.423605,5.0,5.875135,-1.0,23.887325,1.0,54.957125,38.431292,...,4.866311,-1.0,-1.000000,27.447664,1.0,-1.0,"[gender, hematocrit, mcv, ferritin, Inconclusi...",-1.0,5.0,7.0
1355,79.318152,-1.000000,0.326100,5.0,-1.000000,-1.0,-1.000000,1.0,-1.000000,19.034915,...,4.612370,-1.0,-1.000000,-1.000000,1.0,-1.0,"[gender, hematocrit, mcv, ferritin, Inconclusi...",-1.0,5.0,7.0
1776,40.843230,-1.000000,1.920539,5.0,-1.000000,-1.0,-1.000000,1.0,-1.000000,26.487905,...,4.181791,-1.0,0.993699,-1.000000,1.0,-1.0,"[gender, hematocrit, mcv, ferritin, Inconclusi...",-1.0,5.0,7.0
1853,73.596086,-1.000000,1.917797,5.0,-1.000000,-1.0,-1.000000,1.0,-1.000000,27.743841,...,3.653847,-1.0,-1.000000,-1.000000,1.0,-1.0,"[gender, hematocrit, mcv, ferritin, Inconclusi...",-1.0,5.0,7.0
3432,81.154030,31.871723,1.263951,5.0,26.787933,-1.0,3.108248,0.0,84.085324,33.298297,...,3.132010,-1.0,2.566249,205.535421,1.0,-1.0,"[gender, hematocrit, mcv, ferritin, Inconclusi...",-1.0,5.0,7.0


In [94]:
a.describe()[['hemoglobin', 'gender', 'mcv']]

,hemoglobin,gender,mcv
count,32.000000,32.000000,32.000000
mean,9.182157,0.656250,80.514565
std,2.263935,0.482559,0.357676
min,6.083269,0.000000,80.010863
25%,6.959684,0.000000,80.227946
50%,8.874222,1.000000,80.511749
75%,11.297080,1.000000,80.756179
max,12.810431,1.000000,81.550401


In [74]:
df_5.describe()[['hemoglobin', 'gender', 'mcv', 'ret_count']]

,hemoglobin,gender,mcv,ret_count
count,151.000000,151.000000,151.000000,151.000000
mean,9.638535,0.523179,94.926720,4.067254
std,2.033352,0.501125,7.534971,1.162834
min,6.041316,0.000000,80.010863,2.111593
25%,7.907294,0.000000,97.271049,3.128659
50%,9.707347,1.000000,98.610752,3.989234
75%,11.529939,1.000000,99.291808,5.101900
max,12.948876,1.000000,99.927430,5.987249


In [33]:
a1_index_list = list(a1.index)
a1_x_test_df = X_test_df.iloc[a1_index_list]
len(a1_x_test_df)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
a1_x_test_df.describe()

#### 3 - Anemia of chronic disease

In [69]:
acd_test_df = test_df[test_df.y_actual == 3]
acd_x_test_df = X_test_df[X_test_df.label== 3]
combined_acd_test_df = pd.concat([acd_test_df, acd_x_test_df], axis=1)
combined_acd_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
13,5.0,13.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 'A...",3.0,3.0,7.248493,360.445195,...,1,0.814285,126.511825,-1.000000,-1.000000,-1.000000,-1.00000,21.745480,-1.000000,3
16,5.0,16.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 'A...",3.0,3.0,12.474623,485.283040,...,1,1.205270,13.187918,38.154465,19.230585,17.546213,71.58987,37.423868,-1.000000,3
19,5.0,19.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 'A...",3.0,3.0,12.874146,301.420171,...,1,1.594326,113.029862,117.035382,31.349922,27.826310,46.97690,38.622438,33.399927,3
20,5.0,20.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 'A...",3.0,3.0,6.735053,472.029959,...,1,1.255650,127.470359,-1.000000,-1.000000,-1.000000,-1.00000,20.205160,-1.000000,3
29,5.0,29.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 'A...",3.0,3.0,10.736097,212.975715,...,0,1.826583,32.981234,-1.000000,-1.000000,-1.000000,-1.00000,32.208291,-1.000000,3


In [70]:
combined_acd_test_df.y_pred.value_counts()

3.0    1720
4.0      32
7.0      20
Name: y_pred, dtype: int64

In [52]:
df_3 = combined_acd_test_df[combined_acd_test_df.y_pred==4]
df_3.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
354,5.0,354.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'I...",3.0,4.0,8.204232,31.997458,...,0,1.638100,148.984855,59.040314,53.796258,28.329593,51.517073,24.612695,33.744497,3
1416,5.0,1416.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'I...",3.0,4.0,9.540981,32.219941,...,0,1.329128,40.922304,-1.000000,-1.000000,-1.000000,-1.000000,28.622944,-1.000000,3
1603,6.0,1603.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 't...",3.0,4.0,12.965359,71.604228,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,38.896076,-1.000000,3
1834,6.0,1834.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 't...",3.0,4.0,6.878322,89.702400,...,0,1.111510,17.624213,-1.000000,-1.000000,-1.000000,-1.000000,20.634965,-1.000000,3
3891,5.0,3891.0,0.0,-1.0,0.0,"['hematocrit', 'gender', 'mcv', 'ferritin', 'I...",3.0,4.0,9.209679,32.961456,...,0,0.815034,14.812757,110.661002,72.444646,20.705894,85.278586,27.629038,58.178129,3


In [53]:
df_3['trajectory'].value_counts()

['hematocrit', 'gender', 'mcv', 'ferritin', 'Iron deficiency anemia']            12
['hematocrit', 'gender', 'mcv', 'ferritin', 'tibc', 'Iron deficiency anemia']     8
Name: trajectory, dtype: int64

In [58]:
df_3a = df_3[df_3.trajectory =="['hematocrit', 'gender', 'mcv', 'ferritin', 'tibc', 'Iron deficiency anemia']"]

In [59]:
df_3a.describe()[['hemoglobin', 'gender', 'mcv', 'ferritin', 'tibc']]

,hemoglobin,gender,mcv,ferritin,tibc
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,9.659830,0.625000,76.705162,76.059891,442.806018
std,2.324243,0.517549,1.274486,20.724588,3.009252
min,6.878322,0.000000,75.070477,36.721830,438.142730
25%,7.611308,0.000000,75.545498,68.017171,440.673568
50%,9.688312,1.000000,76.855130,79.907712,442.994515
75%,11.341770,1.000000,77.700484,90.895029,445.407447
max,12.965359,1.000000,78.270577,98.906333,446.213024


In [ ]:
acd_hem_pred_df = combined_acd_test_df[(combined_acd_test_df.y_pred==2)]
acd_hem_pred_df.mcv.mean()

In [ ]:
#rows with tibc in their trajectory
tibc_acd_df = pd.DataFrame()
count=0
for i, row in acd_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'tibc' in traj:
        tibc_acd_df = tibc_acd_df.append(row)
tibc_acd_df

In [ ]:
X_test_df.loc[6523]

#### 6 - Aplastic Anemia

In [103]:
aplastic_test_df = test_df[test_df.y_actual == 6]
aplastic_x_test_df = X_test_df[X_test_df.label== 6]

In [104]:
combined_aplastic_test_df = pd.concat([aplastic_test_df, aplastic_x_test_df], axis=1)
combined_aplastic_test_df.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
15,5.0,15.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",6.0,6.0,11.461191,-1.000000,...,0,0.874083,107.025907,-1.000000,51.971137,-1.000000,75.019791,34.383573,-1.0,6
22,5.0,22.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",6.0,6.0,9.349407,-1.000000,...,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,28.048220,-1.0,6
26,5.0,26.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",6.0,6.0,6.382069,-1.000000,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,19.146207,-1.0,6
48,5.0,48.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",6.0,6.0,9.186527,31.803670,...,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,27.559580,-1.0,6
53,5.0,53.0,1.0,1.0,0.0,"['gender', 'hematocrit', 'mcv', 'ret_count', '...",6.0,6.0,8.725146,205.090491,...,1,0.385118,119.244641,116.228638,-1.000000,25.086717,-1.000000,26.175438,-1.0,6


In [105]:
combined_aplastic_test_df.y_pred.value_counts()

6.0    1581
7.0     143
1.0      44
3.0      29
4.0       7
2.0       2
Name: y_pred, dtype: int64

In [106]:
combined_aplastic_test_df.trajectory.value_counts()

['gender', 'hematocrit', 'mcv', 'ret_count', 'Aplastic anemia']                                                                 1441
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'ret_count', 'Aplastic anemia']                                         122
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'Inconclusive diagnosis']                                                57
['gender', 'hematocrit', 'mcv', 'ferritin', 'Inconclusive diagnosis']                                                             48
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'rbc', 'tibc', 'Inconclusive diagnosis']                                 27
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'rbc', 'Vitamin B12/Folate deficiency anemia']                           27
['gender', 'hematocrit', 'mcv', 'ferritin', 'Anemia of chronic disease']                                                          23
['gender', 'hematocrit', 'mcv', 'segmented_neutrophils', 'rbc', 'tibc

In [110]:
df_6 = combined_aplastic_test_df[combined_aplastic_test_df.y_pred==7]
df_6.head()

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred,hemoglobin,ferritin,...,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
84,7.0,84.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",6.0,7.0,6.954617,-1.0,...,0,1.349218,139.103477,-1.000000,48.805154,-1.000000,46.545731,20.863851,-1.0,6
245,7.0,245.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",6.0,7.0,11.529423,-1.0,...,1,0.934370,50.632369,49.817976,45.912342,6.364053,64.467187,34.588268,-1.0,6
259,5.0,259.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",6.0,7.0,8.474117,-1.0,...,1,1.088555,38.563861,34.053703,37.378866,5.360520,66.708818,25.422350,-1.0,6
309,5.0,309.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'ferritin', 'I...",6.0,7.0,12.039245,-1.0,...,1,1.035887,141.844453,82.515434,2.168798,15.952233,49.645805,36.117736,-1.0,6
375,5.0,375.0,0.0,-1.0,1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",6.0,7.0,7.450994,-1.0,...,1,0.208039,26.169009,-1.000000,-1.000000,-1.000000,-1.000000,22.352981,-1.0,6


In [113]:
b = pd.DataFrame()
for i, row in df_6.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'segmented_neutrophils' in row.trajectory:
        b = b.append(row)
b.head()

,cholestrol,copper,creatinine,episode_length,ethanol,ferritin,folate,gender,glucose,hematocrit,...,ret_count,reward,segmented_neutrophils,serum_iron,terminated,tibc,trajectory,tsat,y_actual,y_pred
84,139.103477,-1.000000,1.349218,7.0,48.805154,-1.0,-1.000000,0.0,46.545731,20.863851,...,1.254490,-1.0,5.686044,20.468297,1.0,-1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",-1.0,6.0,7.0
245,50.632369,49.817976,0.934370,7.0,45.912342,-1.0,6.364053,1.0,64.467187,34.588268,...,0.748284,-1.0,6.601030,248.361025,1.0,-1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",-1.0,6.0,7.0
259,38.563861,34.053703,1.088555,5.0,37.378866,-1.0,5.360520,1.0,66.708818,25.422350,...,0.207301,-1.0,-1.000000,245.383427,1.0,-1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",-1.0,6.0,7.0
375,26.169009,-1.000000,0.208039,5.0,-1.000000,-1.0,-1.000000,1.0,-1.000000,22.352981,...,0.297321,-1.0,-1.000000,-1.000000,1.0,-1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",-1.0,6.0,7.0
503,-1.000000,-1.000000,-1.000000,7.0,-1.000000,-1.0,-1.000000,0.0,-1.000000,23.869843,...,0.050107,-1.0,4.345141,-1.000000,1.0,-1.0,"['gender', 'hematocrit', 'mcv', 'segmented_neu...",-1.0,6.0,7.0


In [114]:
b.describe()[['mcv']]

,mcv
count,95.000000
mean,98.814404
std,0.825259
min,96.522969
25%,98.398196
50%,98.955344
75%,99.372110
max,99.991267


In [ ]:
a2 = aplastic_test_df[aplastic_test_df.y_actual != aplastic_test_df.y_pred]
a2.head()

In [ ]:
a2_index_list = list(a2.index)
a2_x_test_df = X_test_df.iloc[a2_index_list]
a2_x_test_df

In [ ]:
X_test_df.describe()

#### 3 - Iron deficiency anemia 

There are a total of 121 IDA test samples. Of these, 3 are diagnosed as hemolytic anemia while 18 are diagnosed as having aplastic anemia 

In [ ]:
#ida_test_df = test_df[test_df.y_pred == 3]
ida_test_df = test_df[test_df.y_actual == 3]
ida_x_test_df = X_test_df[X_test_df.label== 3]
combined_ida_test_df = pd.concat([ida_x_test_df, ida_test_df], axis=1)
combined_ida_test_df.head()

In [ ]:
combined_ida_test_df.mcv.mean()

In [ ]:
#Looking at the 3 that are diagnoses as hemolytic
ida_hem_pred_df = combined_ida_test_df[(combined_ida_test_df.y_pred==1)|(combined_ida_test_df.y_pred==2)]
ida_hem_pred_df.mcv.mean()

In [ ]:
len(ida_hem_pred_df)

In [ ]:
a3 = ida_test_df[ida_test_df.y_actual != ida_test_df.y_pred]
a3.head()

In [ ]:
#rows with tibc in their trajectory
tibc_ida_df = pd.DataFrame()
count=0
for i, row in ida_test_df.iterrows():
    try:
        traj = ast.literal_eval(row.trajectory)
    except:
        traj = row.trajectory
    if 'tibc' in traj:
        tibc_ida_df = tibc_ida_df.append(row)
tibc_ida_df

In [ ]:
b3_index_list = list(tibc_ida_df.index)
b3_x_test_df = X_test_df.iloc[b3_index_list]
b3_x_test_df